Panel is not installed by default in the Pyodide distribution that JupyterLite is built on, therefore we must install it manually:

In [2]:
import micropip

In [3]:
 await micropip.install([
          "https://cdn.holoviz.org/panel/1.5.5/dist/wheels/bokeh-3.6.2-py3-none-any.whl",
          "https://cdn.holoviz.org/panel/1.5.5/dist/wheels/panel-1.5.5-py3-none-any.whl"
        ])

In [4]:
%pip install altair

Once installed we can activate the `pn.extension` as normal.

In [5]:
import panel as pn
pn.extension('vega')

Panel components will now work just like in a regular notebook environment.

In [6]:
w = pn.widgets.FloatSlider(start=0, end=3.14)

pn.Row(w, pn.bind(pn.pane.Str, w))

Row
    [0] FloatSlider(end=3.14)
    [1] ParamFunction(function, _pane=Str, defer_load=False)

Even complex examples relying on custom extensions (e.g. `'vega'`) will work:

In [7]:
import altair as alt
import pandas as pd

df = pd.read_json("https://raw.githubusercontent.com/vega/vega/master/docs/data/penguins.json")

brush = alt.selection_interval(name='brush')  # selection of type "interval"

chart = alt.Chart(df).mark_point().encode(
    x=alt.X('Beak Length (mm):Q', scale=alt.Scale(zero=False)),
    y=alt.Y('Beak Depth (mm):Q', scale=alt.Scale(zero=False)),
    color=alt.condition(brush, 'Species:N', alt.value('lightgray'))
).properties(
    width=250,
    height=250
).add_params(
    brush
)

vega_pane = pn.pane.Vega(chart, debounce=10)

def filtered_table(selection):
    if selection is None:
        return '## No selection'
    query = ' & '.join(
        f'{crange[0]:.3f} <= `{col}` <= {crange[1]:.3f}'
        for col, crange in selection.items()
    )
    return pn.pane.DataFrame(df.query(query), width=600)

pn.Column(
    'Select points on the plot and watch the linked table update.',
    sizing_mode='stretch_width'
).servable()

pn.Row(
    vega_pane,
    pn.Column(
        pn.bind(filtered_table, vega_pane.selection.param.brush),
        scroll=True, width=650, height=300
    )
)

Row
    [0] Vega(Chart, debounce=10, selection=Selection)
    [1] Column(height=300, scroll=True, sizing_mode='fixed', width=650)
        [0] ParamFunction(function, _pane=Markdown, defer_load=False)

The contents of Panelite are updated on every release, however Jupyterlite caches old edits to files. If you want to reset the storage and **delete all modifications you made to the contents** then use the [Reset_Jupyterlite.ipynb](Reset_Jupyterlite.ipynb) notebook.